In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast
import re

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/btp_t5_4epoch/transformers/default/1/config.json
/kaggle/input/btp_t5_4epoch/transformers/default/1/spiece.model
/kaggle/input/btp_t5_4epoch/transformers/default/1/tokenizer.json
/kaggle/input/btp_t5_4epoch/transformers/default/1/tokenizer_config.json
/kaggle/input/btp_t5_4epoch/transformers/default/1/model.safetensors
/kaggle/input/btp_t5_4epoch/transformers/default/1/special_tokens_map.json
/kaggle/input/btp_t5_4epoch/transformers/default/1/generation_config.json


In [2]:
!pip install gliner -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 90.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00


In [3]:
import logging
logging.getLogger('transformers').setLevel(logging.ERROR)

In [4]:
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.3 MB/s eta 0:00:00


In [5]:
import requests
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM 

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32


model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large-ft", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)

config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
import requests
from io import BytesIO
from PIL import Image, UnidentifiedImageError

def extract_text_from_image(image_source):
    try:

        if image_source.startswith("http://") or image_source.startswith("https://"):
            response = requests.get(image_source)
            image = Image.open(BytesIO(response.content))
        else:
            image = Image.open(image_source)
        
        prompt = "<OCR>"
        inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

        # Generate output
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=524,
            num_beams=3,
            do_sample=False
        )

        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        parsed_answer = processor.post_process_generation(generated_text, task="<OCR>", image_size=(image.width, image.height))

        return parsed_answer

    except UnidentifiedImageError:
        return "Error loading image"
    except requests.exceptions.RequestException:
        return "Error retrieving image from URL"
    except Exception as e:
        return f"Unexpected error: {e}"
        

In [74]:
image_path = "https://m.media-amazon.com/images/I/41hHTaag7+L.jpg" 
text_data = extract_text_from_image(image_path)
print("Extracted Text:", text_data)


Extracted Text: {'<OCR>': '400 LBSpressure < 400 lbsBackrestArm Length 14.6"Height29.9"Seat Depth 19.9".Height44.5" -47.4"4.7" seatSeat Width 23.6\'Height19.3" -22.2"Depth 32.5\'Width 28.3\''}


In [75]:
text_data

{'<OCR>': '400 LBSpressure < 400 lbsBackrestArm Length 14.6"Height29.9"Seat Depth 19.9".Height44.5" -47.4"4.7" seatSeat Width 23.6\'Height19.3" -22.2"Depth 32.5\'Width 28.3\''}

In [76]:
import ast

def extract_values(text):
    try:
        
        if isinstance(text, dict):
            return ' '.join(str(value) for value in text.values())
        # If the input is a string, parse it into a dictionary
        data_dict = ast.literal_eval(text)
        if isinstance(data_dict, dict):
            return ' '.join(str(value) for value in data_dict.values())
    except (ValueError, SyntaxError):
        # Handle cases where parsing fails
        return None
    return None

text_data = extract_values(text_data)

In [77]:
text_data

'400 LBSpressure < 400 lbsBackrestArm Length 14.6"Height29.9"Seat Depth 19.9".Height44.5" -47.4"4.7" seatSeat Width 23.6\'Height19.3" -22.2"Depth 32.5\'Width 28.3\''

In [78]:
def clean_text(text):
    if isinstance(text, str):
       
        return re.sub(r'[^a-zA-Z0-9\s.,!?;:()\[\]\'"@#$%&*+-/=_<>|]', '', text)
    return text 

In [79]:
text_data = clean_text(text_data)

In [80]:
text_data  = text_data.lower()

In [81]:
def add_space_before_numbers(text):

    updated_text = re.sub(r'(?<![0-9\s.])(\d)', r' \1', text)
    return updated_text

In [82]:
text_data = add_space_before_numbers(text_data)

In [83]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    if isinstance(text, str): 
        return " ".join(word for word in text.split() if word.lower() not in stop_words)
    return text 

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [84]:
text_data = remove_stopwords(text_data)

In [85]:
def correct_weight_separation(input_text):

    units = [
        "g", "kg", "mg", "ml", "l", "oz", "lb", "pcs", "ct", "cfqty",  
        "gram", "kilogram", "microgram", "milligram", "litre", "liter", "ounce", "pound", "ton", "piece", "pieces" 
    ]
    

    unit_pattern = "|".join(units) 
    pattern = rf"(\d+(\.\d+)?(?:{unit_pattern}))"  
    
    corrected_text = re.sub(pattern, r" \1 ", input_text)

    corrected_text = re.sub(r"\s{2,}", " ", corrected_text).strip()
    
    return corrected_text

In [86]:
text_data = correct_weight_separation(text_data)

In [87]:
text_data

'400 lbspressure < 400 lbsbackrestarm length 14.6"height 29.9"seat depth 19.9".height 44.5" - 47.4" 4.7" seatseat width 23.6\'height 19.3" - 22.2"depth 32.5\'width 28.3\''

In [88]:
def correct_dimension_separation(input_text):

    units = [
        "cm", "mm", "m", "in", "inch", "ft", "yard", 
        "centimetre", "millimetre", "metre", "foot", "yard", "inches"  
    ]   
    unit_pattern = "|".join(units) 
    pattern = rf"(\d+(\.\d+)?(?:{unit_pattern}))"

    corrected_text = re.sub(pattern, r" \1 ", input_text)

    corrected_text = re.sub(r"\s{2,}", " ", corrected_text).strip()
    
    return corrected_text


In [89]:
text_data = correct_dimension_separation(text_data)

In [90]:
def correct_quantity_separation(input_text):

    voltage_units = [
        "kV", "mV", "V", "kilovolt", "millivolt", "volt" , "v" , "kv" , "mv"
    ]
    
    wattage_units = [
        "kW", "W", "kilowatt", "watt" , "w" , "kw"
    ]
    
    volume_units = [
        "cl", "centilitre", "ft³", "cubic foot", "in³", "cubic inch", "cup", "dl", "decilitre",
        "fl oz", "fluid ounce", "gal", "gallon", "imp gal", "imperial gallon", "l", "litre", "litres"
        "µl", "microlitre", "ml", "millilitre", "pt", "pint", "qt", "quart"
    ]
    
    voltage_pattern = r"(\d+(\.\d+)?(?:{}))".format("|".join(voltage_units))
    wattage_pattern = r"(\d+(\.\d+)?(?:{}))".format("|".join(wattage_units))
    volume_pattern = r"(\d+(\.\d+)?(?:{}))".format("|".join(volume_units))

    def add_space_after_unit(pattern, text):
        return re.sub(pattern, r" \1 ", text)
    
    corrected_text = input_text
    corrected_text = add_space_after_unit(voltage_pattern, corrected_text)
    corrected_text = add_space_after_unit(wattage_pattern, corrected_text)
    corrected_text = add_space_after_unit(volume_pattern, corrected_text)
    
    corrected_text = re.sub(r"\s{2,}", " ", corrected_text).strip()
    
    return corrected_text


In [91]:
text_data = correct_quantity_separation(text_data)

In [92]:
text_data

'400 lbspressure < 400 lbsbackrestarm length 14.6"height 29.9"seat depth 19.9".height 44.5" - 47.4" 4.7" seatseat width 23.6\'height 19.3" - 22.2"depth 32.5\'width 28.3\''

In [26]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
model_dir = "/kaggle/input/btp_t5_4epoch/transformers/default/1"
model1 = AutoModelForSeq2SeqLM.from_pretrained(model_dir)


In [28]:
model_name = "google-t5/t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [93]:
def extract_entity_value(group_id, entity_name, text):
    input_text = f"Group ID: {group_id}, Entity Name: {entity_name}, Text: {text}"
    inputs = tokenizer(f"Extract {entity_name} from: {input_text}", return_tensors="pt", max_length=512, truncation=True, padding=True)
    
    device = next(model1.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    outputs = model1.generate(**inputs, max_length=32, num_return_sequences=1, do_sample=False)
    
    outputs = outputs.cpu()
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

model1.to('cuda')

test_group_id = 801829
test_entity_name = "maximum_weight_recommendation"
test_text = text_data
predicted_value = extract_entity_value(test_group_id, test_entity_name, test_text)
print(f"Predicted entity value: {predicted_value}")

Predicted entity value: 400 pound


In [30]:
from gliner import GLiNER

model2 = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

gliner_multitask_performance.png:   0%|          | 0.00/76.8k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/14.5k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.76G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [94]:


labels = ['item_weight', 'item_volume', 'voltage', 'wattage',
       'maximum_weight_recommendation', 'height', 'depth', 'width']

entities = model2.predict_entities(text_data, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

29.9" => height
19.9" => depth
44.5" - 47.4" => height
23.6' => width
19.3" - 22.2" => height
32.5' => depth
28.3' => width
